In [3]:
# 모든 행 출력
pd.set_option('display.max_rows', None)

# 모든 열 출력
pd.set_option('display.max_columns', None)

# 열 너비 자동 조정 (가독성 향상)
pd.set_option('display.width', None)

# 각 컬럼에 표시할 최대 문자 수 늘리기 (딕셔너리 컬럼 등)
pd.set_option('display.max_colwidth', None)


In [1]:
import pandas as pd

# 경로 설정
file_path = "gs://codeit-project/hackle/2023-07-05.parquet"
key_path = "/home/leesh/codeit_project/codeit-project-docker/config/key.json"

# GCS에서 parquet 불러오기
df_230705 = pd.read_parquet(file_path, storage_options={"token": key_path})

# 확인
print(df_230705.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2878668 entries, 0 to 2878667
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Asia/Seoul         object 
 1   event_key          object 
 2   session_id         object 
 3   user_id            object 
 4   value              float64
 5   user_properties    object 
 6   hackle_properties  object 
 7   event_properties   object 
 8   id                 object 
 9   device_id          object 
dtypes: float64(1), object(9)
memory usage: 219.6+ MB
None


In [4]:
df_230705.head()

,Asia/Seoul,event_key,session_id,user_id,value,user_properties,hackle_properties,event_properties,id,device_id
0,2023-07-06T00:00:00.234Z,view_home_tap,GCZf2jHnnpSHH7DnviWmLLiG20j1,GCZf2jHnnpSHH7DnviWmLLiG20j1,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osversion': '15.3.1', 'devicevendor': 'Apple', 'versionname': '1.2.15', 'osname': 'iOS', 'platform': 'Mobile', 'devicemodel': 'iPhone12,8'}",[],f0c6f1b5-2c63-4944-9755-f70f8c116829,19d76555-5bf6-4f86-9d79-e71b4f1e3589
1,2023-07-06T00:00:00.285Z,$session_start,8PZ7GgUaVXSAGn3lPv2E51lfeir1,8PZ7GgUaVXSAGn3lPv2E51lfeir1,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osversion': '16.5.1', 'devicevendor': 'Apple', 'versionname': '1.2.15', 'osname': 'iOS', 'platform': 'Mobile', 'devicemodel': 'iPhone15,2'}",[],f76c51fd-1784-4161-bc9f-8ec621117179,37ba7f82-917f-464d-bfd8-2a83c4aef701
2,2023-07-06T00:00:00.372Z,view_profile_tap,ZeH6snQ1gxWQfRodLFDV6qMMcmG2,ZeH6snQ1gxWQfRodLFDV6qMMcmG2,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osversion': '16.5.1', 'devicevendor': 'Apple', 'versionname': '1.2.15', 'osname': 'iOS', 'platform': 'Mobile', 'devicemodel': 'iPhone12,1'}",[],07a10746-b0cf-465d-9326-ba43ea02721c,3fe79d88-9d5c-43ff-b7ae-513855f9e97f
3,2023-07-06T00:00:00.397Z,launch_app,8PZ7GgUaVXSAGn3lPv2E51lfeir1,8PZ7GgUaVXSAGn3lPv2E51lfeir1,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osversion': '16.5.1', 'devicevendor': 'Apple', 'versionname': '1.2.15', 'osname': 'iOS', 'platform': 'Mobile', 'devicemodel': 'iPhone15,2'}",[],6e09b3a6-690b-4204-a8aa-91fb71b68863,37ba7f82-917f-464d-bfd8-2a83c4aef701
4,2023-07-06T00:00:00.473Z,$session_start,raovhvckyKM8m1X2hKrIja8yIb92,raovhvckyKM8m1X2hKrIja8yIb92,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osversion': '16.0.2', 'devicevendor': 'Apple', 'versionname': '1.2.15', 'osname': 'iOS', 'platform': 'Mobile', 'devicemodel': 'iPhone12,8'}",[],37d76708-46b3-46eb-a6a3-3c29d68bb8ca,25cf1eb3-65b8-418b-b99b-db2f87ef647a


In [5]:
df_simple = df_230705.copy()

# 딕셔너리 → Series로 분해 (중요한 컬럼만)
df_simple['language'] = df_simple['hackle_properties'].apply(lambda x: x.get('language'))
df_simple['devicemodel'] = df_simple['hackle_properties'].apply(lambda x: x.get('devicemodel'))


# 1. UTC 기준으로 파싱
df_simple['event_datetime'] = pd.to_datetime(df_simple['Asia/Seoul'], utc=True)

# 2. 한국시간(Asia/Seoul)으로 타임존 변환
df_simple['event_datetime'] = df_simple['event_datetime'].dt.tz_convert('Asia/Seoul')

# 최종 확인용 컬럼만 선택
df_simple = df_simple[['event_datetime','device_id', 'event_key', 'devicemodel', 'language']]

In [6]:
df_simple['event_datetime'] = df_simple['event_datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_simple.head()

,event_datetime,device_id,event_key,devicemodel,language
0,2023-07-06 09:00:00,19d76555-5bf6-4f86-9d79-e71b4f1e3589,view_home_tap,"iPhone12,8",ko-KR
1,2023-07-06 09:00:00,37ba7f82-917f-464d-bfd8-2a83c4aef701,$session_start,"iPhone15,2",ko-KR
2,2023-07-06 09:00:00,3fe79d88-9d5c-43ff-b7ae-513855f9e97f,view_profile_tap,"iPhone12,1",ko-KR
3,2023-07-06 09:00:00,37ba7f82-917f-464d-bfd8-2a83c4aef701,launch_app,"iPhone15,2",ko-KR
4,2023-07-06 09:00:00,25cf1eb3-65b8-418b-b99b-db2f87ef647a,$session_start,"iPhone12,8",ko-KR


In [7]:
df_simple.to_parquet("/home/leesh/codeit_project/codeit-project-docker/parquet/230705.parquet", engine="pyarrow", compression="snappy")